In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import os,sys
from mask_to_submission import *

In [2]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches


foreground_threshold = 0.25

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

patch_size = 16

# Loaded a set of images
root_dir = "training/"
gt_dir = root_dir + "groundtruth/"

files = os.listdir(gt_dir)
gt_imgs = [load_image(gt_dir + files[i]) for i in range(len(files))]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])


root_dir = "predict_training/"
gt_dir = root_dir + "groundtruth/"

files = os.listdir(gt_dir)
gt_imgs2 = [load_image(gt_dir + files[i]) for i in range(len(files))]
gt_patches2 = [img_crop(gt_imgs2[i], patch_size, patch_size) for i in range(len(gt_imgs2))]
gt_patches2 =  np.asarray([gt_patches2[i][j] for i in range(len(gt_patches2)) for j in range(len(gt_patches2[i]))])

foreground_threshold = 0.25

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])
Y = (Y==0)*1  #reverts the groundtruth (black must be road to be consistent)
Z = np.asarray([value_to_class(np.mean(gt_patches2[i])) for i in range(len(gt_patches2))])
score = f1_score(Y,Z,average='weighted')
accuracy = accuracy_score(Y,Z)

print('F-score on training dataset = '+str(score))
print('Accuracy on training dataset = '+str(accuracy))

F-score on training dataset = 0.924031166052
Accuracy on training dataset = 0.924592
